In [1]:
import base64
import time

from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as expected
from selenium.webdriver.support.wait import WebDriverWait

In [2]:
url = 'http://rage/?som=274&show_dist=on&xaxis=branch&xaxis=build_number&xaxis=build_tag&v_product=XenServer&v_product=XenServer-Transformer&v_soms=274&v_numvms=125&v_numvdis=0&v_machine_name=xrtuk-08-01&v_machine_name=xrtuk-08-02&v_machine_name=xrtuk-08-03&v_machine_name=xrtuk-08-04&v_srtype=ext:/dev/sdb&v_nousb=t&v_optimise=t&v_xdtailor=t&v_pvdrivers=t&v_vmtype=win7sp1-x86&v_vmram=1024&v_experiment=vmrun_cron&v_vmcron=0:25:waitvmstart&v_vmcooloff=0&v_dom0_tunevcpus=&v_vmindex_sorted_by_numvms_finished=125&v_branch=ely&v_branch=release/falcon/master&v_branch=release/inverness/master&v_branch=release/jura/master&v_build_number=137272&v_build_number=15&v_build_number=56&v_build_number=76&v_build_number=8&v_build_number=88&v_build_number=99&v_build_tag=all_hotfixes,https://repo.citrite.net/xs-local-assembly/xenserver/release/inverness/master/14/XS73.iso&v_build_tag=all_hotfixes,https://repo.citrite.net/xs-local-assembly/xenserver/release/jura/master/40/XS74.iso&v_build_tag=all_hotfixes,https://repo.citrite.net/xs-local-assembly/xenserver/release/jura/master/58/XS74.iso&v_build_tag=all_hotfixes,https://repo.citrite.net/xs-local-assembly/xenserver/release/jura/master/59/XS74.iso&v_build_tag=all_hotfixes,https://repo.citrite.net/xs-local-assembly/xenserver/release/jura/master/68/XS74.iso&v_build_tag=all_hotfixes,https://repo.citrite.net/xs-local-assembly/xenserver/release/jura/master/76/XS74.iso&v_build_tag=all_hotfixes,https://repo.citrite.net/xs-local-assembly/xenserver/release/jura/master/98/XS74.iso&v_build_tag=all_hotfixes,https://repo.citrite.net/xs-local-assembly/xenserver/release/jura/master/99/XS74.iso&v_label=Inverness#brief_report_analysis'

In [3]:
options = Options()
options.add_argument('-headless')
driver = Firefox(executable_path='C:\dev\geckodriver', firefox_options=options)
wait = WebDriverWait(driver, timeout=300)
driver.get(url)
wait.until(expected.visibility_of_element_located((By.TAG_NAME, 'canvas')))

canvas = driver.find_element_by_tag_name('canvas')
canvas_bytes = canvas.screenshot_as_png

In [4]:
with open(r"canvas.png", 'wb') as f:
    f.write(canvas.screenshot_as_png)

In [29]:
driver.quit()

In [9]:
base64.b64encode(canvas.screenshot_as_png) == canvas.screenshot_as_base64

TypeError: string argument without an encoding

In [7]:
base64.b64encode(canvas.screenshot_as_png)

b'...'

In [8]:
canvas.screenshot_as_base64

'...'